THIS IS ONLY A FIRST DRAFT VERSION -- IT IS NOT TUNED AND CHECKED YET!

To do:
- simplify maxpooling and flattening
- more tuning, improve performance for low alpha
- perhaps try different initializer
- visualize reflection weights

In [ ]:
import sys
sys.path.append('../src/')

import matplotlib.pyplot as plt
%matplotlib inline

import keras
from keras.layers import MaxPooling2D, Dense, Flatten

import utils
import reflection_layer as RL

In [ ]:
sample_settings = {'add_blob': None,
                   'alpha': 0.5,
                   'image_size': 32,
                   'noise_mean' : 1,
                   'noise_std' : 0.1,
                   'blob_size': 16, 
                   'blob_fill_value': 0,
                   'blob_mean': 0,    
                   'blob_delta': 1}

In [ ]:
# set up a non-sequential NN
im_size = sample_settings['image_size']
inp = keras.layers.Input(shape=(im_size, im_size, 1))
r1 = RL.ReflectionLayer()(inp)
m1 = MaxPooling2D(pool_size=(im_size, im_size))(r1)
r2 = RL.ReflectionLayer()(inp)
m2 = MaxPooling2D(pool_size=(im_size, im_size))(r2)
ma = keras.layers.Maximum()([m1,m2])
fl = Flatten()(ma)
out = Dense(1, activation="sigmoid")(fl)
model = keras.models.Model(inputs=inp, outputs=out)
model.summary()

In [ ]:
# define generators for training and validation data
train_gen = utils.yield_samples(128, **sample_settings)
valid_gen = utils.yield_samples(16, **sample_settings)

In [ ]:
# train model
model.compile(optimizer=keras.optimizers.Adam(),
              loss="binary_crossentropy",
              metrics=["accuracy"])
training_history = model.fit_generator(train_gen,
                                       steps_per_epoch=1,
                                       validation_data=valid_gen,
                                       validation_steps=1,
                                       epochs=100)

In [ ]:
# plot change of accuracy during training
keys = ['val_accuracy', 'accuracy']
for k in keys:
    plt.plot(training_history.history[k])
plt.xlabel('Epoch')
plt.legend(keys, loc='lower left')
plt.show()